In [1]:
!pip install seaborn==0.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 208 kB 5.4 MB/s 
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.11.2
    Uninstalling seaborn-0.11.2:
      Successfully uninstalled seaborn-0.11.2


In [2]:
# from google.colab import drive
# drive.mount('/gdrive')

In [3]:
%ls drive

MyDrive/


In [4]:
%cd drive/MyDrive/'Colab Notebooks'/'Alura - validacao'/

/content/drive/MyDrive/Colab Notebooks/Alura - validacao


In [5]:
from modelos import Modelos 
from modelos import Explorador

In [6]:
import pandas as pd
import numpy as np

In [7]:
URI="https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
df = pd.read_csv(URI).drop(columns=["Unnamed: 0"], axis=1)
df['modelo'] = df.idade_do_modelo + np.random.randint(-2, 3, size=10000)
df.modelo = df.modelo + abs(df.modelo.min()) + 1

In [8]:
x = df[["preco", "idade_do_modelo", "km_por_ano", "modelo"]]
y = df["vendido"]

SEED = 158020

In [9]:
dados_azar = df.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano", "modelo"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
4999,74023.29,0,12,24812.80412,14
5322,84843.49,0,13,23095.63834,15
5319,83100.27,0,19,36240.72746,23
5316,87932.13,0,16,32249.56426,19
5315,77937.01,0,15,28414.50704,17


In [10]:
modelos = Modelos(SEED, x, y)
# explorador = Explorador(SEED, espaco_de_parametros, x, y)

In [11]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold

#  cvStratifiedKFold = StratifiedKFold(**kwargs)
cvKFold           = KFold(n_splits = 10)
cvGroupKFold      = GroupKFold(n_splits = 10)

In [12]:
import seaborn as sns
import matplotlib as plt

# Árvore de Decisão - Explorando hiper parâmetros em 2 dimensões

In [13]:
def busca_2d(min_samples_leafs):
  resultados = []
  for max_depth in range(1,33):
    for min_samples_leaf in min_samples_leafs:
      tabela = modelos.arvore_decisao_2d(cvKFold, df.modelo, max_depth, min_samples_leaf)
      resultados.append(tabela)
  resultados = pd.DataFrame(resultados)
  return resultados


In [14]:
resultados_2d = busca_2d([32, 64, 128, 256])
resultados_2d.head()

,MAX_DEPTH,MIN_SAMPLES_LEAF,train_score,test_score
0,1,32,75.791111,75.78
1,1,64,75.791111,75.78
2,1,128,75.791111,75.78
3,1,256,75.791111,75.78
4,2,32,75.791111,75.78


In [15]:
resultados_2d.sort_values("test_score", ascending=False).head()

,MAX_DEPTH,MIN_SAMPLES_LEAF,train_score,test_score
10,3,128,78.751111,78.69
15,4,256,78.751111,78.69
14,4,128,78.751111,78.69
13,4,64,78.751111,78.69
11,3,256,78.751111,78.69


In [16]:
corr = resultados_2d.corr()

In [17]:
# sns.set(style="white")

# # Generate a mask for the upper triangle
# mask = np.zeros_like(corr, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

# # Set up the matplotlib figure
# f, ax = plt.subplots(figsize=(11, 9))

# # Generate a custom diverging colormap
# cmap = sns.diverging_palette(220, 10, as_cmap=True)

# # Draw the heatmap with the mask and correct aspect ratio
# sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [18]:
resultados_2d = busca_2d([128, 192, 256, 512])
resultados_2d.head()

,MAX_DEPTH,MIN_SAMPLES_LEAF,train_score,test_score
0,1,128,75.791111,75.78
1,1,192,75.791111,75.78
2,1,256,75.791111,75.78
3,1,512,75.791111,75.78
4,2,128,75.791111,75.78


In [19]:
import matplotlib.pyplot as plt

In [20]:
resultados_2d.sort_values("test_score", ascending=False).head()

,MAX_DEPTH,MIN_SAMPLES_LEAF,train_score,test_score
13,4,192,78.751111,78.69
12,4,128,78.751111,78.69
8,3,128,78.751111,78.69
9,3,192,78.751111,78.69
10,3,256,78.751111,78.69


In [21]:
corr = resultados_2d.corr()

In [22]:
# sns.set(style="white")

# # Generate a mask for the upper triangle
# mask = np.zeros_like(corr, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

# # Set up the matplotlib figure
# f, ax = plt.subplots(figsize=(11, 9))

# # Generate a custom diverging colormap
# cmap = sns.diverging_palette(220, 10, as_cmap=True)

# # Draw the heatmap with the mask and correct aspect ratio
# sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})

# Árvore de Decisão - Explorando hiper parâmetros em 3 dimensões

In [23]:
def busca_3d(min_samples_leafs, min_samples_splits):
  resultados = []
  for max_depth in range(1,33):
    for min_samples_leaf in min_samples_leafs:
      for min_samples_split in min_samples_splits:
        tabela = modelos.arvore_decisao_3d(cvKFold, df.modelo, max_depth, min_samples_leaf, min_samples_split)
        resultados.append(tabela)
  resultados = pd.DataFrame(resultados)
  return resultados

In [24]:
resultados_3d = busca_3d([32, 64, 128, 256], [32, 64, 128, 256])
resultados_3d.head()

,MAX_DEPTH,MIN_SAMPLES_LEAF,train_score,test_score,fit_time,score_time
0,1,32,75.791111,75.78,0.007227,0.001996
1,1,32,75.791111,75.78,0.006879,0.002049
2,1,32,75.791111,75.78,0.006699,0.002083
3,1,32,75.791111,75.78,0.007031,0.002246
4,1,64,75.791111,75.78,0.007312,0.002301


In [25]:
resultados_3d.sort_values("test_score", ascending=False).head()

,MAX_DEPTH,MIN_SAMPLES_LEAF,train_score,test_score,fit_time,score_time
55,4,64,78.751111,78.69,0.015135,0.002164
56,4,128,78.751111,78.69,0.014520,0.002202
54,4,64,78.751111,78.69,0.014698,0.002230
53,4,64,78.751111,78.69,0.014684,0.002323
52,4,64,78.751111,78.69,0.015017,0.002259


In [26]:
corr = resultados_3d.corr()

In [27]:
# sns.set(style="white")

# # Generate a mask for the upper triangle
# mask = np.zeros_like(corr, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

# # Set up the matplotlib figure
# f, ax = plt.subplots(figsize=(11, 9))

# # Generate a custom diverging colormap
# cmap = sns.diverging_palette(220, 10, as_cmap=True)

# # Draw the heatmap with the mask and correct aspect ratio
# sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})


# Comparando GridSearchCV com RandomizedSearch, busca aleatória

## Grid

In [28]:
import time
from scipy.stats import randint
from sklearn.model_selection import cross_val_score

In [29]:
def imprime_score(scores):
  media = scores.mean() * 100
  desvio = scores.std() * 100
  print("Accuracy médio %.2f" % media)
  print("Intervalo [%.2f, %.2f]" % (media - 2 * desvio, media + 2 * desvio))

In [30]:
espaco_de_parametros = {
    "n_estimators" : [10, 100],
    "max_depth" : [3, 5],
    "min_samples_split" : [32, 64, 128],
    "min_samples_leaf" : [32, 64, 128],
    "bootstrap" : [True, False],
    "criterion" : ["gini", "entropy"]
}

tic = time.time()
explorador = Explorador(SEED, espaco_de_parametros, x_azar, y_azar)
busca = explorador.busca_grid(KFold(n_splits = 5, shuffle=True), x.modelo)
tac = time.time()

print((tac - tic) * (-1))

resultado = pd.DataFrame(busca.cv_results_)
resultado.head()

-305.54680609703064


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.055688,0.001680,0.005480,0.000118,True,gini,3,32,32,10,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.7780,0.7805,0.7630,0.7600,0.7750,0.7713,0.008244,136
1,0.499005,0.011999,0.029796,0.003401,True,gini,3,32,32,100,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.7920,0.7810,0.7595,0.7675,0.7765,0.7753,0.011165,94
2,0.052638,0.002545,0.005878,0.001030,True,gini,3,32,64,10,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.7685,0.7810,0.7745,0.7670,0.7890,0.7760,0.008167,86
3,0.509275,0.015949,0.029832,0.002380,True,gini,3,32,64,100,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.7920,0.7815,0.7595,0.7660,0.7765,0.7751,0.011443,99
4,0.055376,0.002586,0.005550,0.000377,True,gini,3,32,128,10,"{'bootstrap': True, 'criterion': 'gini', 'max_...",0.8030,0.7860,0.7640,0.7655,0.7355,0.7708,0.022752,138


In [31]:
resultados_ordenados_pela_media = resultado.sort_values("mean_test_score", ascending=False)
for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  print("%.3f +-(%.3f) %s" % (linha.mean_test_score, linha.std_test_score*2, linha.params))

0.787 +-(0.022) {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 100}
0.787 +-(0.022) {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 32, 'n_estimators': 100}
0.787 +-(0.022) {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 128, 'n_estimators': 100}
0.787 +-(0.023) {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 32, 'n_estimators': 100}
0.787 +-(0.022) {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 128, 'n_estimators': 100}


In [32]:
tic = time.time()
scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))
tac = time.time()
tempo_passado = tac - tic
print("Tempo %.2f segundos" % tempo_passado)

imprime_score(scores)
melhor = busca.best_estimator_
print(melhor)

Tempo 1264.94 segundos
Accuracy médio 78.68
Intervalo [77.21, 80.15]
RandomForestClassifier(bootstrap=False, max_depth=5, min_samples_leaf=64,
                       min_samples_split=64)


## Random

In [33]:
espaco_de_parametros = {
    "n_estimators" :randint(10, 101),
    "max_depth" : randint(3, 6),
    "min_samples_split" : randint(32, 129),
    "min_samples_leaf" : randint(32, 129),
    "bootstrap" : [True, False],
    "criterion" : ["gini", "entropy"]
}

tic = time.time()
explorador = Explorador(SEED, espaco_de_parametros, x_azar, y_azar)
busca = explorador.busca_random_iter(KFold(n_splits = 20, shuffle=True), 80)
tac = time.time()

print((tac - tic) * (-1))

resultado = pd.DataFrame(busca.cv_results_)
resultado.head()

-761.0466265678406


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,...,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,split18_test_score,split19_test_score,mean_test_score,std_test_score,rank_test_score
0,0.553751,0.005601,0.011918,0.001535,False,gini,5,50,113,58,...,0.796,0.752,0.776,0.766,0.768,0.802,0.764,0.7868,0.018745,4
1,0.456026,0.012992,0.008186,0.000430,False,entropy,5,113,43,35,...,0.798,0.752,0.778,0.766,0.760,0.800,0.764,0.7853,0.018014,20
2,0.367694,0.004587,0.011100,0.000622,True,gini,4,95,95,57,...,0.798,0.758,0.776,0.768,0.758,0.790,0.764,0.7824,0.017716,35
3,0.263298,0.018869,0.008680,0.000709,True,gini,4,85,52,40,...,0.800,0.754,0.760,0.766,0.758,0.798,0.756,0.7800,0.019860,46
4,0.497470,0.008057,0.010518,0.000514,False,entropy,3,114,122,55,...,0.802,0.756,0.760,0.764,0.754,0.788,0.756,0.7755,0.019666,53


In [34]:
resultados_ordenados_pela_media = resultado.sort_values("mean_test_score", ascending=False)
for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  print("%.3f +-(%.3f) %s" % (linha.mean_test_score, linha.std_test_score*2, linha.params))

0.787 +-(0.037) {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 65, 'min_samples_split': 127, 'n_estimators': 57}
0.787 +-(0.036) {'bootstrap': True, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 34, 'min_samples_split': 121, 'n_estimators': 81}
0.787 +-(0.037) {'bootstrap': True, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 38, 'min_samples_split': 119, 'n_estimators': 68}
0.787 +-(0.037) {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 50, 'min_samples_split': 113, 'n_estimators': 58}
0.787 +-(0.037) {'bootstrap': True, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 44, 'min_samples_split': 62, 'n_estimators': 66}
